In [21]:
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import time
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

print("PIL 라이브러리 import 완료!")

#로딩을 표현한 함수
def loading_dot():
    for i in range(3):
        time.sleep(1)
        print('. ', end='')
    print('')
    time.sleep(1)

#이미지 리사이즈
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

# 학습 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서 리사이즈
image_dir_path = os.getenv("HOME") + "/aiffel/RSP_Game/Scissor"
resize_images(image_dir_path)
print("가위 이미지 resize 완료!")
loading_dot()
image_dir_path = os.getenv("HOME") + "/aiffel/RSP_Game/Rock"
resize_images(image_dir_path)
print("바위 이미지 resize 완료!")
loading_dot()
image_dir_path = os.getenv("HOME") + "/aiffel/RSP_Game/Paper"
resize_images(image_dir_path)
print("보 이미지 resize 완료!")
loading_dot()

#데이터를 학습시킨다.
def load_data(img_path, number_of_data=900):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/Scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/Rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/Paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/RSP_Game"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))
loading_dot()

#네트워크 설계
#네트워크 채널 파라메터
n_channel_1 = 32
n_channel_2 = 64
n_dense = 64
n_train_epoch = 10

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3, 3), activation = 'relu', input_shape = (28, 28, 3)))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Conv2D(n_channel_2, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation = 'relu'))
model.add(keras.layers.Dense(3, activation = 'softmax'))
model.summary()

#네트워크 학습
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs=n_train_epoch)

# 테스트 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/RSP_Game/Test/scissor"
resize_images(image_dir_path)
print("테스트 가위 이미지 resize 완료!")
loading_dot()
image_dir_path = os.getenv("HOME") + "/aiffel/RSP_Game/Test/rock"
resize_images(image_dir_path)
print("테스트 바위 이미지 resize 완료!")
loading_dot()
image_dir_path = os.getenv("HOME") + "/aiffel/RSP_Game/Test/paper"
resize_images(image_dir_path)
print("테스트 보 이미지 resize 완료!")
loading_dot()

def load_test_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("실험데이터(x_test)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path_test = os.getenv("HOME") + "/aiffel/RSP_Game/Test"
(x_test, y_test)=load_test_data(image_dir_path_test)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))
loading_dot()

# 측정
test_loss, test_accuracy = model.evaluate(x_test_norm,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

# 프로젝트를 진행하며 어려웠던 점
# ==> 가장 어려웠던 점은 "내가 지금 하고 있는 것이 옳은가?"처럼 방향성에 대한 의문이었다. 그리고 값이 나오게 되는 진행 상황을 잘 이해
#     하지 못하고 코드의 흐름을 읽지 못해 어려움을 겪었다.
#
# 프로젝트를 진행하며 모호한 점
# ==> 학습정확도값이 0.6을 넘지 않아 많은 시도를 해보았는데 같은 조건에서 훈련과 테스트를 시행했을 때 할 때마다 값이 달라지고 
#     그 값이 값이 어느정도 오차범위인 경우도 있었지만 그렇지 않고 큰 격차가 나는 경우도 있었다.
#
# 루브릭 평가 지표를 맞추기 위해 시도한 것들
# ==> 처음에 300장을 훈련시키고 테스트를 했을때 값이 기준치에 미치지 않아 처음에는 훈련 데이터의 양을 늘려 1500장으로 늘려 보기도 하고 
#     파라메터 값을 변경하여 뉴런의 수나 훈련되는 횟수를 변경하기도 해보았으나 조금씩 오르는 듯 보였으나 값이 튀는 경우가 생겨 안정성에
#     문제가 있다고 생각하여 값을 초기화 하였고 모델을 새로 맞춰볼까 생각해 봤지만 지금까지 해보았던 경험이 아까운 마음에 실행에 옮기지는
#     못했다. 그러나 결론적으로 값을 안정적이고 정확하게 만들어 준 것은 명확한 훈련 데이터였다. 이전에 사용했던 데이터는 있는 그대로의
#     (침대와 창문과 같은 여러 사물이 있는) 배경을 사용하였지만 다른 분들의 데이터와 같이 배경을 단색으로 하고 좀 더 명확한 손 데이터 
#     를 사용하여 정확도를 높일 수 있었다. 물론 이것이 완벽한 해결 방안이 아니라고 생각하기에 나중에라도 시간이 된다면 다시 고민해 보고싶다.
#
# 자기 다짐
# ==> 이번 프로젝트를 하며 정해지지는 않았지만 내 스스로 속박되고 시간에 쫓겨 시야가 좁아져 평소라면 생각해 볼 수 있었던 것들도 쉽게 떠올리지
#     못했다. 그리고 노드를 진행하며 깊게 생각해 보지 않는 나를 보게 되었는데 더 꼼꼼하고 질문을 많이 던져보며 앞으로의 프로젝트와 노드를
#     진행할 것이다. 그리고 앞의 노드들을 보며 이해하지 않고 결과만 보고 넘어갔었기에 코드의 한 줄 한 줄이 의미하는 것을 생각하며 공부할 것입니다.

PIL 라이브러리 import 완료!
300  images to be resized.
300  images resized.
가위 이미지 resize 완료!
. . . 
300  images to be resized.
300  images resized.
바위 이미지 resize 완료!
. . . 
300  images to be resized.
300  images resized.
보 이미지 resize 완료!
. . . 
학습데이터(x_train)의 이미지 개수는 900 입니다.
x_train shape: (900, 28, 28, 3)
y_train shape: (900,)
. . . 
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 5, 5, 64)          0         
___________________________________________________